In [1]:
import cPickle

# PARAMS
log_dir = "/home/sforesti/avakas/scratch/sforestier001/logs/CogSci2017/2017-01-17_12-08-55-COGSCI"


filename = log_dir + '/results/competence.pickle'
with open(filename, 'r') as f:
    data_competence = cPickle.load(f)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

config_list = ["RMB"]
n_iter = 100
n_goals = 1000
trial_list = range(1,n_iter + 1) 
config_name = "RMB"


data_gavd = {}

for trial in trial_list:
    data_gavd[trial] = {}
    for region in [1, 2, 3]:
        data_gavd[trial][region] = {}
        for toy in ["toy1", "toy2", "toy3"]:
            data_gavd[trial][region][toy] = {}
            data_gavd[trial][region][toy]["comp_error"] = np.mean([data_competence[config_name][trial]["eval_results"][region][i][toy]["comp_error"] for i in range(n_goals)])
            data_gavd[trial][region][toy]["arm_dist"] = np.mean([data_competence[config_name][trial]["eval_results"][region][i][toy]["arm_dist"] for i in range(n_goals)])
            data_gavd[trial][region][toy]["diva_dist"] = np.mean([data_competence[config_name][trial]["eval_results"][region][i][toy]["diva_dist"] for i in range(n_goals)])

In [9]:
data_avg = {}
for region in [1, 2, 3]:
    data_avg[region] = {}
    data_avg[region]["comp_error"] = np.mean([data_gavd[trial][region][toy]["comp_error"] for toy in ["toy1", "toy2", "toy3"] for trial in trial_list])
    data_avg[region]["arm_dist"] = np.mean([data_gavd[trial][region][toy]["arm_dist"] for toy in ["toy1", "toy2", "toy3"] for trial in trial_list])
    data_avg[region]["diva_dist"] = np.mean([data_gavd[trial][region][toy]["diva_dist"] for toy in ["toy1", "toy2", "toy3"] for trial in trial_list])
    
data_avg

{1: {'arm_dist': 0.27202058397949697,
  'comp_error': 0.30960549459667552,
  'diva_dist': inf},
 2: {'arm_dist': 0.46009236195021347,
  'comp_error': 0.72234074203546283,
  'diva_dist': inf},
 3: {'arm_dist': 0.65821960736571117,
  'comp_error': 0.98775296064881146,
  'diva_dist': inf}}